In [0]:
%load_ext autoreload
%autoreload 2


In [0]:
# carregamento de bibliotecas
import sys
import os
from datetime import date, timedelta

root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

if root not in sys.path:
    sys.path.append(root)


from src.features.utils import *

In [0]:
# construção da tabela histórica de volume de açúcar não refinado exportado
TABELA_INCREMENTAL_RAW = "`api-forecasting`.bronze.volume_exportacao_acucar_incremental"

data_atual = date.today().strftime('%Y-%m')

try:
    df = volume_acucar_exportacao(mes_ano_inicio=data_atual, mes_ano_fim=data_atual)

    # transformação do dataset no formato spark
    df = criar_tabela_spark(df)

except Exception as e:
    raise print('Dado ainda não disponível para captura')


In [0]:
df.show()

In [0]:
# persistência da tabela na camada bronze
df = salvar_tabela_delta_spark(df, 
                         mode='append', partitionby=['date'],merge_schema=True, path=TABELA_INCREMENTAL_RAW)

In [0]:
%sql
-- TABELA CATALOGADA NA CAMADA BRONZE
SELECT * 
FROM `api-forecasting`.bronze.volume_exportacao_acucar_incremental
LIMIT 10